In [2]:
import spacy
nlp_full = spacy.load('en_core_web_lg')

In [30]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

In [37]:
helpful_reviews= pd.read_pickle('/Users/yishu/Documents/insight/helpful_reviews.p')

In [38]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['my', 'your', 'you', 'whole', 'one', 'love','like','tried','great','good','can','could'
                   ,'leave','found','maybe'])

In [39]:
processed_docs = helpful_reviews['r_review'].map(preprocess)
#processed_docs[:10]

dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

0 absorb
1 better
2 climate
3 combination
4 film
5 greasy
6 leaves
7 living
8 looking
9 morning
10 night


In [40]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 4),
 (14, 1),
 (15, 1),
 (16, 1)]

In [45]:
r_review_helpful = review_helpful['r_review']
docs = [nlp_full(desc) for desc in r_review_helpful]

In [46]:
docs[:5]

[I have combination skin that tends to be dry due to living in a dry climate. I found that I could not use this in the morning before the day because it leaves my skin looking greasy. While it did not leave a film, it did not absorb well into my skin. Maybe it would be OK to use at night, but there are better options that will do more for your skin for the price.,
 Truly a holy grail product. Every time I run out, I get antsy and try something new...and I always regret it and come running back to this cream. It feels so good on skin, sinks in pretty quickly and never pills when layering serums or oils at night. This keeps my skin’s texture so smooth and absolutely softens the appearance of fine lines. I’ve got a few best friends hooked, too.,
 This cream feels great on your skin, smells great and is super hydrating. Unfortunately it made me break out. And I’m almost 60 years old. So not great for me.,
 Despite the hefty price tag, I do think this is one of the better moisturizers out t

In [41]:
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in stop_words and len(token) > 3:
            result.append(token)
    return result

In [42]:
def generate_preferred_spelling_dict(words):
    '''
    For some set of words, returns a dict mapping each unique lowercased word to
    its most popular spelling and total occurrences of all spellings.
    '''
    spellings = {}
    for word in words:
        word_lower = word.lower()
        if word_lower in spellings:
            spellings[word_lower].append(word)
        else:
            spellings[word_lower] = [word]
    preferred_spellings = {}
    for (word, spelling_cands) in spellings.items():
        n_occurrences = len(spelling_cands)
        preferred_spelling = max(set(spelling_cands), key=spelling_cands.count)
        preferred_spellings[word] = (preferred_spelling, n_occurrences)
    return preferred_spellings

def generate_multiplicity_dict(words):
    '''
    Counts the number of occurrences of each word, case-sensitive.
    '''
    multiplicities = {}
    for word in words:
        if word not in multiplicities:
            multiplicities[word] = words.count(word)
    return multiplicities


In [43]:
def cands_directly_describing_subject(cands, subj_descriptors):
    return [cand for cand in cands if cand.lower() in subj_descriptors]

In [44]:
def highest_multiplicity_cand(cands, preferred_spellings):
    return max(cands, key=lambda cand: preferred_spellings[cand.lower()][1])

In [51]:
listing_noun_phrases = []
stop_tags = ['PRP', 'DT','IN','TO','MD'] #'IN'
for doc in docs:
    #spans = [span for span in list(doc.noun_chunks) ]
    #tokens = [(token, token.tag_) for span in spans for token in span]
    #print(tokens)
    noun_phrases = []
    for np in doc.noun_chunks:
        if np.root.tag_ not in stop_tags:
            important_descriptors = [word for word in np if not word.tag_ in stop_tags and not word.text == np.root.text]
            noun_phrases.append((important_descriptors, np.root.text))
    listing_noun_phrases.append(noun_phrases)
#listing_noun_phrases

In [52]:
listing_noun_phrase_subjects = [np for listing in listing_noun_phrases for (descriptors, np) in listing]
subject_preferred_spellings = generate_preferred_spelling_dict(listing_noun_phrase_subjects)
popular_descriptors = list(subject_preferred_spellings.items())

In [53]:
popular_descriptors.sort(key=lambda desc: desc[1][1], reverse=True)
#popular_descriptors

In [54]:
most_popular_descriptors = [descriptor for (descriptor, _) in popular_descriptors[:10]]
aggregate_indirect_descriptors = []
indirect_descriptor_phrases = {descriptor:[] for descriptor in most_popular_descriptors}
for listing in listing_noun_phrases:
    for descriptors, subject in listing:
        subject_lower = subject.lower()
        if subject_lower in most_popular_descriptors:
            subject_descriptions = []
            description_buffer = []
            for descriptor in descriptors:
                #if len(descriptor.text) == 1 and re.findall('[^A-Za-z0-9]', descriptor.text): continue
                description_buffer.append(descriptor.text)
                aggregate_indirect_descriptors.append(descriptor.text)
                #print(descriptor.text)
                # If the descriptor directly modifies the subject of the NP, take it
                # and all descriptors in the buffer (that presumably modify this new descriptor)
                if descriptor.head.text == subject:
                    subject_descriptions.append(description_buffer)
                    description_buffer = []
            indirect_descriptor_phrases[subject_lower].append(subject_descriptions)
            
            #print(subject)        
            #print(subject_descriptions)
            
preferred_descriptor_spellings = generate_preferred_spelling_dict(aggregate_indirect_descriptors)
#print(indirect_descriptor_phrases)
# for subject, listings in indirect_descriptor_phrases.items():
#     for i, listing in enumerate(listings):
#         for j, description in enumerate(listing):
#             print(description)
#             for k, descriptor in enumerate(description):
#                 #print(descriptor)
#                 preferred_spelling = preferred_descriptor_spellings[descriptor.lower()][0]
#                 if descriptor != preferred_spelling:
#                     indirect_descriptor_phrases[subject][i][j][k] = preferred_spelling
# print(indirect_descriptor_phrases)
top_indirect_descriptors = {descriptor:[] for descriptor in most_popular_descriptors}
for subject, listings in indirect_descriptor_phrases.items():
    flattened_indirect_descriptor_phrase_list = []
    for listing in listings:
        for description in listing:
            # This will unfortunately put spaces around hyphens, and that sort of thing
            text_description = ' '.join([preferred_descriptor_spellings[descriptor.lower()][0] for descriptor in description])
            flattened_indirect_descriptor_phrase_list.append(text_description)
    preferred_descriptions = list(generate_multiplicity_dict(flattened_indirect_descriptor_phrase_list).items())
    preferred_descriptions.sort(key=lambda desc: desc[1], reverse=True)
    top_indirect_descriptors[subject] = preferred_descriptions

for feature, descriptors in top_indirect_descriptors.items():
    print(f'{feature}:')
    for descriptor, mult in descriptors[:8]:
        print(f'\t{descriptor} ({mult})')

skin:
	my (4143)
	your (550)
	dry (426)
	sensitive (331)
	oily (324)
	combination (232)
	, (152)
	very dry (69)
product:
	great (133)
	my (45)
	amazing (36)
	best (36)
	good (33)
	first (32)
	new (31)
	only (26)
brush:
	my (152)
	best (86)
	great (77)
	blush (68)
	foundation (61)
	powder (52)
	good (43)
	lip (36)
face:
	my (1644)
	your (342)
	whole (45)
	entire (35)
	full (21)
	dry (12)
	their (10)
	small (6)
scent:
	, (71)
	my (58)
	light (57)
	nice (48)
	fresh (41)
	floral (26)
	beautiful (26)
	strong (24)
one:
	second (24)
	my (23)
	new (18)
	best (11)
	good (11)
	original (10)
	not (9)
	first (8)
makeup:
	my (472)
	eye (100)
	your (76)
	all (47)
	face (20)
	waterproof (16)
	full (7)
	other (7)
cream:
	eye (239)
	night (112)
	BB (87)
	my (83)
	hand (39)
	face (37)
	day (32)
	best (25)
products:
	other (165)
	my (70)
	fresh (40)
	many (36)
	Philosophy (33)
	their (31)
	Dior (31)
	Shiseido (27)
time:
	first (118)
	long (109)
	same (65)
	my (47)
	hard (31)
	night (19)
	very long (17)
	